In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sparse

!pip install tabulate
from tabulate import tabulate

%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

from bs4 import BeautifulSoup # Превращалка html в текст.
import re # Регулярные выражения.

from IPython.display import clear_output

import math # Корень квадратный.

import csv


#pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.height', 1000)
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 5000)
pd.set_option('max_colwidth', 800)

'''
Подсказки 
https://www.kaggle.com/c/vacancy-classification-sf01/discussion/50676
http://rusvectores.org/ru/models/
https://www.kaggle.com/c/vacancy-classification-sf01-1/leaderboard
https://github.com/nadbordrozd/blog_stuff/blob/master/classification_w2v/benchmarking_python3.ipynb


'''

'\nПодсказки \nhttps://www.kaggle.com/c/vacancy-classification-sf01/discussion/50676\nhttp://rusvectores.org/ru/models/\nhttps://www.kaggle.com/c/vacancy-classification-sf01-1/leaderboard\nhttps://github.com/nadbordrozd/blog_stuff/blob/master/classification_w2v/benchmarking_python3.ipynb\n\n\n'

In [3]:
df_train = pd.read_csv('train.csv', sep='\t') #, encoding='utf-8'
df_test = pd.read_csv('test.csv', sep='\t') #, encoding='utf-8'
#df_other = pd.read_csv('other.csv', sep='\t') #, encoding='utf-8'

In [4]:
# для теста
#df_train = df_train[:100]
#df_test = df_test[:100]

In [5]:
df_train.head(20)

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уручье),"<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) приглашаем на постоянную работу руководителя секции (хозтовары).</strong></p> <p><strong>Обязанности: </strong></p> <p> </p> <ul> <li>организация эффективного и культурного обслуживания покупателей</li> <li>организация приемки-сдачи товаров на склад</li> <li>консультация покупателей по вопросам, касающимся оказываемых услуг</li> <li>контроль своевременной подачи товаров в торговую секцию</li> <li>проверка качества, сроков годности ТМЦ, наличие маркировок, ценников на товарах</li> <li>организация бесперебойной работы товарной секции и участка склада, закрепленного за ней</li> <li>контроль за сохранностью товаров, торгового оборудования и прочих материальных ценностей</li> <li>инвентаризация товаров</li> </ul> <p> </p> <p><strong>Требования:</s...",1
1,1,Наладчик станков и манипуляторов с ПУ,"Обязанности:работа на токарных станках с ЧПУ Takisana 108 Turn-i Cnjqrf Fanuc, на токарно-фрезерном станке с ЧПУ стойка Fanuc series 21 Oi=ТВ, с опытом работы на освоении деталей <p>Требовния: опыт работы на станочном оборудовании, внимательность, 4-5 разряд</p> <p> </p> <p>Условия: 12 часовой рабочий день, посменно, 1 неделя в день, 2 недели в ночь, график 6/1</p> <p>Проживание в общежитии, при заселении иметь с собой комплект постельного белья, посуду для личного пользования.</p> <p>Оформление по ТК РФ, наличие флюорографии, выплата з/п 15 и 30 числа каждого месяца</p> <p>​​​​​​​</p>",0
2,2,Разработчик С++ (Криптограф),"<strong>Требования:</strong> <ul> <li>Опыт программирования на С++</li> <li>Знание базовых алгоритмов, на которых основывается работа ПО, обеспечивающего шифрование и создание/проверку ЭЦП (SHA, MD5, RSA, PKCS-стандарты, сертификаты, удостоверяющие центры и т.п.)</li> <li>Знание и понимание базовых алгоритмов computer science.</li> </ul> <p> </p> <p><strong>Условия:</strong></p> <ul> <li>Стабильная, развивающаяся софт-компания в Железнодорожном районе.</li> <li>Сильная команда профессионалов.</li> <li>Интересные и разнообразные задачи.</li> <li>Возможность обучения и повышения профессионального уровня.</li> <li>Зарплата обсуждается по итогам собеседования, «белая».</li> <li>Официальное трудоустройство, полный соц. пакет (оплачиваемые отпуска и больничные, пенсионные отчисления в полном...",0
3,3,Фрезеровщик,"<p>Условия:</p> <ul> <li>На работу вахтовым методом в г. Калуга требуются фрезеровщики, вахта 60/30 (45/30).</li> <li>Оплата фиксированная и стабильная 190 руб./час, 11 час./день, 6 дн./нед. (заработная плата 54500 руб./мес).</li> <li>Проживание, спец. одежда предоставляются, питание самостоятельно.</li> <li>Выдача аванса производится каждые 2 недели.</li> <li>Прямой работодатель, официальное трудоустройство по ТК РФ.</li> </ul> <p>Требования:</p> <ul> <li>От 4 разряда, опыт от 3 лет.</li> </ul> <p>Универсально- фрезерные станки.</p>",0
4,4,Мерчендайзер/продавец-консультант,"<p><strong>Компания Палладиум Стандарт - признанный лидер продаж на рынке дверной фурнитуры приглашает на работу энергичных сотрудников на открытые вакансии мерчендайзер/продавец-консультант для работы в магазине партнерских сетей , отдел замочно - скобяных изделий:</strong></p> <p> </p> <p>Леруа Мерлен Косино (ул. Святоозерская, 1А, ТРЦ Косино Парк).</p> <p>ОБИ Варшавка, (Варшавское ш., 97, ТЦ Ритейл Парк)</p> <p> </p> <p><strong>Обязанности:</strong></p> <ul> <li>Контроль поставки и приемки товара на складе в сети (контроль вывоза товара в зал)</li> <li>Выкладка продукции на полки (контроль ценников, товарного вида, маркировки и т.п.)</li> <li>Консультация покупателей</li> <li>Составление заказов исходя из потребностей и остатков на складе</li> <li>Возвраты (сбор неликвидного товара)...",1
5,5,Мастер по эксплуатации зданий,"<p><strong>Обязанности:</strong></p> <ul> <li><strong>Обеспечение бесперебойной работы сетей водоснабжения и водоотведения, отопления и вентиляции.</strong></li> <li><strong>Проведение текущих и капит

In [6]:
df_train[df_train['target'].isin([1])].head(20)

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уручье),"<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) приглашаем на постоянную работу руководителя секции (хозтовары).</strong></p> <p><strong>Обязанности: </strong></p> <p> </p> <ul> <li>организация эффективного и культурного обслуживания покупателей</li> <li>организация приемки-сдачи товаров на склад</li> <li>консультация покупателей по вопросам, касающимся оказываемых услуг</li> <li>контроль своевременной подачи товаров в торговую секцию</li> <li>проверка качества, сроков годности ТМЦ, наличие маркировок, ценников на товарах</li> <li>организация бесперебойной работы товарной секции и участка склада, закрепленного за ней</li> <li>контроль за сохранностью товаров, торгового оборудования и прочих материальных ценностей</li> <li>инвентаризация товаров</li> </ul> <p> </p> <p><strong>Требования:</s...",1
4,4,Мерчендайзер/продавец-консультант,"<p><strong>Компания Палладиум Стандарт - признанный лидер продаж на рынке дверной фурнитуры приглашает на работу энергичных сотрудников на открытые вакансии мерчендайзер/продавец-консультант для работы в магазине партнерских сетей , отдел замочно - скобяных изделий:</strong></p> <p> </p> <p>Леруа Мерлен Косино (ул. Святоозерская, 1А, ТРЦ Косино Парк).</p> <p>ОБИ Варшавка, (Варшавское ш., 97, ТЦ Ритейл Парк)</p> <p> </p> <p><strong>Обязанности:</strong></p> <ul> <li>Контроль поставки и приемки товара на складе в сети (контроль вывоза товара в зал)</li> <li>Выкладка продукции на полки (контроль ценников, товарного вида, маркировки и т.п.)</li> <li>Консультация покупателей</li> <li>Составление заказов исходя из потребностей и остатков на складе</li> <li>Возвраты (сбор неликвидного товара)...",1
6,6,Торговый представитель,"<em><strong>НА СТАБИЛЬНОЕ И РАСПРОСТРАНЕННОЕ НАПРАВЛЕНИЯ &quot;ЗАМОРОЗКИ&quot; ТРЕБУЮТСЯ ТОРГОВЫЕ ПРЕДСТАВИТЕЛИ</strong></em> <p><strong>Обязанности</strong>:</p> <ul> <li>расширение клиентской базы,</li> <li>заключение договоров,</li> <li>контроль ДЗ</li> <li>увеличение ассортимента</li> </ul> <p> </p> <p><strong>Требования</strong>:</p> <ul> <li><strong><em>опыт работы в активных продажах приветствуется,</em></strong></li> <li>навыки ведения переговоров,</li> <li>коммуникабельность, активность, огромное желание работать.</li> </ul> <p> </p> <p><strong>Условия</strong>:</p> <ul> <li>работа по Воронежу,</li> <li>работа в динамично развивающейся компании, являющейся лидером рынка,</li> <li>перспектива карьерного и профессионального роста,</li> <li>оформление по ТК РФ</li> <li>компенсаци...",1
7,7,Торговый представитель,<strong>Обязанности:</strong> <ul> <li>Функционал работы торгового представителя</li> <li>Выполнения плана продаж</li> <li>Увеличение активной клиентской базы</li> <li>Системность работы по маршрутам</li> <li>Продвижение и расширение ассортимента компании на закрепленной территории</li> </ul> <p><strong>Требования:</strong></p> <ul> <li>Наличие опыта не есть обязательным условием</li> </ul> <strong>Условия:</strong> <ul> <li>5-ти дневный рабочий день</li> <li>гибкий график посещения офиса</li> <li>компенсационный пакет</li> </ul>,1
8,8,Менеджер по продажам,"<p><strong>Обязанности:</strong></p> <ul> <li>Продажа садовой, лесопарковой техники, бензо и электроинструмента, консультация посетителей,предпродажная подготовка,погрузочно-разгрузочные работы.</li> <li>Ведение отчетности.</li> </ul> <p><br /><strong>Требования:</strong></p> <ul> <li>Знание ПК - 1С.,базовые программы, знание кассовой дисциплины.</li> <li>Коммуникабельность, ответственность, грамотная речь, неконфликтность.</li> <li>Опыт приветствуется.</li> </ul> <p><br /><strong>Условия:</strong></p> <ul> <li>Собеседование проходит после рассмотрения резюме.</li> <li>График работы:5/2 . 9:00-19:00.</li> <li>Официальное трудоустройство.</li> </ul>",1
14,14,Менеджер по работе с клиентами,"<p><em>Друзья! Сейчас мы находимся в поиске инициативных и предприимчивых в нашу команду.</em></p> <p>Несколько слов о компании. Более 5 лет мы работае

In [7]:
df_train[df_train['name']=='Менеджер по продажам'].head(20)

,id,name,description,target
8,8,Менеджер по продажам,"<p><strong>Обязанности:</strong></p> <ul> <li>Продажа садовой, лесопарковой техники, бензо и электроинструмента, консультация посетителей,предпродажная подготовка,погрузочно-разгрузочные работы.</li> <li>Ведение отчетности.</li> </ul> <p><br /><strong>Требования:</strong></p> <ul> <li>Знание ПК - 1С.,базовые программы, знание кассовой дисциплины.</li> <li>Коммуникабельность, ответственность, грамотная речь, неконфликтность.</li> <li>Опыт приветствуется.</li> </ul> <p><br /><strong>Условия:</strong></p> <ul> <li>Собеседование проходит после рассмотрения резюме.</li> <li>График работы:5/2 . 9:00-19:00.</li> <li>Официальное трудоустройство.</li> </ul>",1
27,27,Менеджер по продажам,"<p><strong><em>Федеральная мониторинговая компания &quot;ТН-ГРУПП&quot;</em></strong></p> <p><strong>Один из ведущих интеграторов систем мониторинга транспорта и стационарных объектов, имеющая в своем составе собственное производство.</strong></p> <p><strong>Компания динамично развивается и набирает команду профессионалов. Если ты готов покорять новые вершины, быть частью молодого и успешного коллектива - присоединяйся!</strong></p> <p> </p> <p><strong>Требования:</strong></p> <ul> <li>Высшее образование</li> <li>Опыт работы в продажах</li> <li>Опыт &quot;холодных звонков&quot;</li> <li>Высокая обучаемость, нацеленность на результат</li> </ul> <p> </p> <p><strong>Функции:</strong></p> <ul> <li>Поиск и привлечение клиентов (В2В)</li> <li>Проработка мониторинговых решений под задачи клие...",1
38,38,Менеджер по продажам,"<p> </p> <p><strong>Обязанности:</strong></p> <ul> <li>Поиск и привлечение клиентов</li> <li>Взаимодействие с автодилерами</li> <li>Формирование, выполнение плана продаж</li> <li>Аналитика конкурентной среды</li> <li>Проведение переговоров</li> <li>Заключение договоров</li> <li>Формирование необходимой отчетности</li> </ul> <p> </p> <p><strong>Требования:</strong></p> <ul> <li>Образование н/высшее, высшее</li> <li>Опыт работы от 2 лет на аналогичной должности</li> <li>Отличное знание техники продаж</li> <li>Желательно наличие водительского удостоверения категории &quot;B&quot;, автомобиля</li> </ul> <p> </p> <p><strong>Условия:</strong></p> <ul> <li>Высокий уровень заработной платы, оклад 45 000 +% от продаж</li> <li>Компенсация ГСМ, мобильная связь</li> <li>Возможность профессионально...",1
51,51,Менеджер по продажам,"<p><strong>Московская компания открывает региональный отдел продаж.</strong></p> <p> </p> <p><strong>Обязанности</strong>:</p> <ul> <li>поиск клиентов, переговоры, сбор и обработка заявок.</li> <li>консультации по телефону, онлайн консультация на сайте, помощь в подборе оборудования.</li> </ul> <p><strong>Требования</strong>:</p> <ul> <li>ответственность, коммуникабельность</li> <li>целеустремленность,</li> <li><strong>работа на результат</strong>.</li> </ul> <p><strong>Приветствуются</strong>:</p> <ul> <li>опыт продаж,</li> <li>знание 1С бухгалтерии,</li> <li>наличие личного автомобиля</li> </ul> <p><strong>Условия</strong>:</p> <ul> <li>работа в офисе, удаленная работа</li> <li>большой % от продаж</li> <li>возможна еженедельная выплата заработной платы</li> </ul>",1
68,68,Менеджер по продажам,"<p><strong>Обязанности</strong>:</p> <ul> <li> <p>активный поиск клиентов</p> </li> <li> <p>рассылка коммерческих предложений</p> </li> <li> <p>ведение переговоров</p> </li> <li> <p>заключение договоров</p> </li> <li> <p>прием и обработка заказов</p> </li> <li> <p>выполнение плана продаж</p> </li> </ul> <p><strong>Требования:</strong></p> <ul> <li>Успешный опыт продаж услуг .</li> <li>Желание работать и зарабатывать.</li> </ul> <p><strong>Условия:</strong></p> <ul> <li>Постоянная, полный день (5/2), работа на территории работодателя. Оформление, согласно ТК.</li> <li>Оформление, отпуск, больничный и пр. по ТК</li> <li>Оплата сотовой связи</li> </ul>",1
94,94,Менеджер по продажам,"<p><strong>СРОЧНАЯ ВАКАНСИЯ Менеджера по продажам !</strong></p> <p> </p> <p>Хочешь работать в здоро

In [8]:
df_train.name.unique()

array(['Заведующий отделом/секцией в магазин YORK (Уручье)',
       'Наладчик станков и манипуляторов с ПУ',
       'Разработчик С++ (Криптограф)', ...,
       'Специалист по качеству и пищевой безопасности',
       'Специалист по тестированию ПО (Fraud Monitoring)',
       'Торговый представитель в Алматы'], dtype=object)

In [9]:
import pymystem3
mystem = pymystem3.Mystem()

import pymorphy2
m2 = pymorphy2.MorphAnalyzer()

In [10]:
# mystem.lemmatize разбивает "PKCS-стандарты", С++, C#
def my_tokenizer_mystem(text):
    lemms = mystem.lemmatize(text)
    #return [x for x in lemms if len(x)>2]
    # буквы, плюсы (С++), дефисы (PKCS-стандарты), числа в конце (EC2)
    return [x for x in lemms if re.match('^[A-zА-яЁё\#\+\-]+\d*$', x)]

def my_tokenizer(text):
    pattern = re.compile(r'[A-zА-яЁё\#\+\-]+\d*')
    return [m2.normal_forms(word)[0] for word in  pattern.findall(text) if len(word) > 1]


my_tokenizer("test; Текстж текСты пывап 34ва жлвоа,EC2, PKCS-стандарты,C++ ;C#")


['test',
 'текстж',
 'текст',
 'пывап',
 'ва',
 'жлвоа',
 'ec2',
 'pkcs-стандарт',
 'c++',
 'c#']

In [11]:
def tokenized(df_input):
    vectorized = CountVectorizer(ngram_range=(1,2), tokenizer=my_tokenizer)
    #vectorized = TfidfTransformer(norm=None, smooth_idf=False)
    
    #оставим разреженную матрицу (sparse как результат)
    vectorized_matrix = vectorized.fit_transform(df_input["all_texts"])
    
    obj_output = {
        'matrix' : vectorized_matrix,
        'items' : vectorized.vocabulary_.items(),
        'sample' : df_input['sample'],
        'target' : df_input['target']
    }
    
    return obj_output;
    
    '''
    vectorized_items = vectorized.vocabulary_.items()
    vectorized_matrix = vectorized_matrix.toarray()
    
    df_output = pd.DataFrame(
        vectorized_matrix,
        #np.asarray(vectorized_matrix),
        columns=[x[0] for x in sorted(vectorized_items, key=lambda x: x[1])]
    )
    
    df_output['name'] = df_input['name']
    df_output['sample'] = df_input['sample']
    df_output['target'] = df_input['target']
    
    return df_output;
    '''
    

def preproc_data(df_input):    
    df_output = (df_input.copy()
                 # Удаляем идентификатор
                .drop('id', axis=1)
                .assign(                    
                    texts_without_tags = lambda df: df['description'].map(lambda x: BeautifulSoup(x, 'lxml').text),
                    all_texts = lambda df: df['name'] + ', ' + df['texts_without_tags'],
                )
                # it converts to object
                .pipe(tokenized)
                #.pipe(pd.get_dummies, drop_first=True, columns=['name'])
    )        
    
    return df_output

In [ ]:
df_train.loc[:, 'sample'] = 'train'
df_test.loc[:, 'sample'] = 'test'
df_ = df_test.append(df_train, sort=False).reset_index(drop=True)


obj_preproc = df_.pipe(preproc_data)

#print(obj_preproc)

#df_train_preproc = df_preproc.query('sample == "train"').drop(['sample'], axis=1).reset_index(drop=True)
#df_test_preproc = df_preproc.query('sample == "test"').drop(['sample'], axis=1).reset_index(drop=True)

#print(df_train_preproc.shape)
#print(df_test_preproc.shape)

In [294]:
#извлечение строк в разреженной матрице
# indices - индексы извлекаемых строк, N - количество строк! в исходной матрице
def extractor(indices, N):
    indptr=np.arange(len(indices)+1)
    data=np.ones(len(indices))
    shape=(len(indices),N)
    return sparse.csr_matrix((data,indices,indptr), shape=shape, dtype=np.int64)

In [297]:
E_test.shape

(170179, 3310400)

In [298]:
obj_preproc['matrix'].shape

(370179, 3310400)

In [302]:
E_test = extractor(np.linspace(0,len(df_test)-1,len(df_test),dtype='int'), obj_preproc['matrix'].shape[0])
E_train = extractor(np.linspace(len(df_test),obj_preproc['matrix'].shape[0]-1,len(df_train),dtype='int'), obj_preproc['matrix'].shape[0])

m_test_preproc = E_test*obj_preproc['matrix']
m_train_preproc = E_train*obj_preproc['matrix']

#m_test_preproc = obj_preproc['matrix'].A[:len(df_test)]
#m_train_preproc = obj_preproc['matrix'].A[len(df_test):]
m_train_y = obj_preproc['target'][len(df_test):]

In [303]:
obj_preproc['matrix']

<370179x3310400 sparse matrix of type '<class 'numpy.int64'>'
	with 80260818 stored elements in Compressed Sparse Row format>

In [304]:
X_train, y_train = m_train_preproc, m_train_y
X_test, y_test = m_test_preproc, []

In [305]:
print(X_train.shape, y_train.shape)

(200000, 3310400) (200000,)


In [ ]:
'''

svc = Pipeline([
    #("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), 
    ("linear svc", SVC(kernel="linear"))
])

svc_tfidf = Pipeline([
    ("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), 
    ("linear svc", SVC(kernel="linear"))
])

all_models = [    
    ("svc", svc),
    ("svc_tfidf", svc_tfidf),
]

unsorted_scores = [(name, model, cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc').mean()) for name, model in all_models]


scores = sorted(unsorted_scores, key=lambda x: -x[-1])


print (tabulate(scores, floatfmt=".4f", headers=("model", 'score')))

'''

In [ ]:
model_SVC = SVC(kernel="linear", probability=True)

In [ ]:
cross_val = cross_val_score(model_SVC, X_train, y_train, cv=5, scoring='roc_auc', n_jobs=-1, verbose=10).mean()
print(cross_val)

In [ ]:
model_SVC.fit(X_train, y_train)
Y_test_SVC_proba = model_SVC.predict_proba(X_test)

In [ ]:
result_SVC = df_test.loc[:,('id',)]
#берем единички
result_SVC['target'] = list(map(lambda x: x[1], Y_test_SVC_proba))
result_SVC.to_csv('result_SVC.csv', index=False)

In [ ]:
# TODO
#оптимизация
